# Classification of hand-written digits

Start by loading the tensorflow package

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import gzip
import os
from six.moves import urllib
tf.logging.set_verbosity(tf.logging.ERROR)

The following functions are needed to load the data. Just execute the cell below, do not bother about all the details.

In [2]:
SOURCE_URL = 'https://storage.googleapis.com/cvdf-datasets/mnist/'
WORK_DIRECTORY = 'data'
IMAGE_SIZE = 28
NUM_CHANNELS = 1
NUM_LABELS = 10
def maybe_download(filename):
  """Download the data from Yann's website, unless it's already here."""
  if not tf.gfile.Exists(WORK_DIRECTORY):
    tf.gfile.MakeDirs(WORK_DIRECTORY)
  filepath = os.path.join(WORK_DIRECTORY, filename)
  if not tf.gfile.Exists(filepath):
    filepath, _ = urllib.request.urlretrieve(SOURCE_URL + filename, filepath)
    with tf.gfile.GFile(filepath) as f:
      size = f.size()
    print('Successfully downloaded', filename, size, 'bytes.')
  return filepath


def extract_data(filename, num_images):
  """Extract the images into a 4D tensor [image index, y, x, channels].
  Values are rescaled from [0, 255] down to [-0.5, 0.5].
  """
  print('Extracting', filename)
  with gzip.open(filename) as bytestream:
    bytestream.read(16)
    buf = bytestream.read(IMAGE_SIZE * IMAGE_SIZE * num_images * NUM_CHANNELS)
    data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
    data = data.reshape(num_images, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS)
    return data


def extract_labels(filename, num_images):
  """Extract the labels into a vector of int64 label IDs."""
  print('Extracting', filename)
  with gzip.open(filename) as bytestream:
    bytestream.read(8)
    buf = bytestream.read(1 * num_images)    
    labels = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)    
    # one hot encode
    onehot_encoded = list()
    for value in labels:
        letter = [0 for _ in range(0,NUM_LABELS)]
        letter[value] = 1
        onehot_encoded.append(letter)    
    labels_onehot_encoded  = np.array(onehot_encoded)
    return labels_onehot_encoded

Download, extract and load the mnist data

In [3]:
# Get the data.
train_data_filename = maybe_download('train-images-idx3-ubyte.gz')
train_labels_filename = maybe_download('train-labels-idx1-ubyte.gz')
test_data_filename = maybe_download('t10k-images-idx3-ubyte.gz')
test_labels_filename = maybe_download('t10k-labels-idx1-ubyte.gz')

# Extract it into numpy arrays.
train_data = extract_data(train_data_filename, 60000)
train_labels = extract_labels(train_labels_filename, 60000)
test_data = extract_data(test_data_filename, 10000)
test_labels = extract_labels(test_labels_filename, 10000)

# Dataset size
train_size = train_labels.shape[0]
test_size = test_labels.shape[0]

# Normalize the data
train_data = train_data/255.0
test_data = test_data/255.0

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


### The model

Define placeholders for the data. The input data Xim are $28\times 28$ pixels grayscale images. The first dimension None will be the number of data points, determined when launching the graph. This placeholder is flattend into a matrix with $28\times 28=784$ columns, where each colum represents one pixel. The placeholder for the output data Y is a one hot encoding representing the 10 different classes. One hot encoding for the class "0" is [1 0 0 0 0 0 0 0 0 0], for class "1" is [0 1 0 0 0 0 0 0 0 0] and so forth.

In [4]:
# The placeholder for the input
Xim = tf.placeholder(dtype = tf.float32, shape = [None, 28, 28, 1])

# Number of data points (in the present batch)
batchsize = tf.shape(Xim)[[0]]

# Flatten the data into a matrix with  28 x 28 = 784 columns
X = tf.reshape(tensor = Xim, shape = [batchsize,784 ])

# The placeholder of the output. 
Y = tf.placeholder(dtype = tf.float32, shape = [None, 10])


Define the weights of the network

In [10]:
input_data = 784
M1 = 4 #number of channels
M2 = 8
M3 = 12
M4 = 200 #number of hidden units
classes = 10

W1 = tf.Variable(initial_value = tf.truncated_normal([5,5,1,M1], stddev = 0.1))
b1 = tf.Variable(initial_value = tf.ones(M1)/M1)

W2 = tf.Variable(initial_value = tf.truncated_normal([5,5,M1,M2], stddev = 0.1))
b2 = tf.Variable(initial_value = tf.ones(M2)/M2)

W3 = tf.Variable(initial_value = tf.truncated_normal([4,4,M2,M3], stddev = 0.1))
b3 = tf.Variable(initial_value = tf.ones(M3)/M3)

W4 = tf.Variable(initial_value = tf.truncated_normal([7*7*M3, M4], stddev = 0.1))
b4 = tf.Variable(initial_value = tf.ones(M4)/M4)

W5 = tf.Variable(initial_value = tf.truncated_normal([M4, classes], stddev = 0.1))
b5 = tf.Variable(initial_value = tf.ones(classes)/classes)

Define the actual network model

In [11]:
# The network model
stride1 = 1
stride2 = 2
H1 = tf.nn.relu(tf.nn.conv2d(Xim,W1,strides=[1,stride1,stride1,1], padding='SAME') + b1)
H2 = tf.nn.relu(tf.nn.conv2d(H1,W2,strides=[1,stride2,stride2,1], padding='SAME') + b2)
H3 = tf.nn.relu(tf.nn.conv2d(H2,W3,strides=[1,stride2,stride2,1], padding='SAME') + b3)
H3flat = tf.reshape(tensor = H3, shape = [batchsize, 7*7*M3])
H4 = tf.nn.relu(tf.matmul(H3flat, W4) + b4)
Z = tf.matmul(H4, W5) + b5
P = tf.nn.softmax(Z)

### The training 
Define the loss function and the training parameters

In [12]:
# The loss function: Compute the cross-entropy
#cross_entropy = -tf.reduce_mean(tf.reduce_sum(Y * tf.log(P), reduction_indices=1))
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = Z, labels = Y)
cross_entropy = tf.reduce_mean(cross_entropy)

# Define the training
gamma = 0.002 # The learning rate
train_step = tf.train.AdamOptimizer(gamma).minimize(cross_entropy)

Accuracy of the trained model. 0 is worst and 1 is the best.
These lines are for evaluating the performance. Don't bother to much about them!

In [13]:
prediction = tf.argmax(P, 1) # In each row of Q, determine the index of the column with highest probability
true = tf.argmax(Y, 1) 
correct_prediction = tf.equal(true, prediction) # Compare with true labels. TRUE if correct, FALSE if not.
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


Launch the graph and train it!

In [16]:
# Create session and initialize  variables
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Initialize the test and training error resutls
test_accuracy = []
test_cross_entropy = []
test_iter = []
train_accuracy = []
train_cross_entropy = []
train_iter = []

BATCH_SIZE = 100 # Batch size
NUM_ITERATIONS = 4000 # Total number of iterations
iter_in_one_epoch = np.floor_divide(train_size,BATCH_SIZE) # Number of iterations in each epoch
# The main training loop
for t in range(0, NUM_ITERATIONS+1):
    
    # The iteration number in the present epoch
    j = (t%iter_in_one_epoch)
    
    # If a new epoch, reshuffle the data indices
    if j == 0:
        train_ind = np.random.choice(train_size,size=train_size, replace=False)        
        
    # Compute the indices of the batch
    offset = j*BATCH_SIZE
    batch_ind = train_ind[offset:(offset+BATCH_SIZE)]
    
    # Extract a batch with BATCH_SIZE images and labels
    batch_xs = train_data[batch_ind, ...]
    batch_ys = train_labels[batch_ind,]        
    
    # Do one gradient setp
    sess.run(train_step,feed_dict = {Xim: batch_xs, Y: batch_ys})

    # Evaluate the performance on training data
    # These lines are for evaluating the performance. Don't bother too much about them!

    if (t%10 == 0):        
        # Evaluate on test data
        train_tmp = sess.run([accuracy,cross_entropy], feed_dict = {Xim: batch_xs, Y: batch_ys})
        train_accuracy.append(train_tmp[0])
        train_cross_entropy.append(train_tmp[1])
        train_iter.append(t)

    # Evaluate the performance on training data at every 100th itertation
    # These lines are for evaluating the performance. Don't bother too much about them!

    if (t%100==0):
        # Evaluate on test data
        test_tmp = sess.run([accuracy,cross_entropy], feed_dict = {Xim: test_data, Y: test_labels})
        test_accuracy.append(test_tmp[0])
        test_cross_entropy.append(test_tmp[1])
        test_iter.append(t)

        # Also print iteration number and prediction accuracy
        print("Step %d: train acc. %.2f  train cross-entropy %.2f test acc. %.3f  test cross-entropy %.3f" % (t,train_tmp[0],train_tmp[1],test_tmp[0],test_tmp[1]))



Step 0: train acc. 0.15  train cross-entropy 2.26 test acc. 0.089  test cross-entropy 2.323
Step 100: train acc. 0.94  train cross-entropy 0.19 test acc. 0.929  test cross-entropy 0.246
Step 200: train acc. 0.98  train cross-entropy 0.09 test acc. 0.954  test cross-entropy 0.148
Step 300: train acc. 0.96  train cross-entropy 0.24 test acc. 0.959  test cross-entropy 0.124
Step 400: train acc. 0.98  train cross-entropy 0.08 test acc. 0.967  test cross-entropy 0.099
Step 500: train acc. 0.97  train cross-entropy 0.08 test acc. 0.973  test cross-entropy 0.089
Step 600: train acc. 0.97  train cross-entropy 0.08 test acc. 0.976  test cross-entropy 0.075
Step 700: train acc. 0.97  train cross-entropy 0.07 test acc. 0.975  test cross-entropy 0.079
Step 800: train acc. 0.99  train cross-entropy 0.06 test acc. 0.978  test cross-entropy 0.076
Step 900: train acc. 0.99  train cross-entropy 0.05 test acc. 0.980  test cross-entropy 0.062
Step 1000: train acc. 0.99  train cross-entropy 0.03 test acc.

### The evaluation

The remaining code produces the plots needed to evaluate the training and prediction

In [ ]:
# Do the plot of cross entropy
plt.plot(train_iter, train_cross_entropy,'b-', label='Training data (mini-batch)')
plt.plot(test_iter, test_cross_entropy,'r-', label='Testing data')
plt.xlabel('Iteration')
plt.ylabel('Cross-entropy')
plt.ylim([0,min(test_cross_entropy)*3])
plt.title('Cross-entropy')
plt.grid(True)
plt.legend(loc='best')
plt.show()

# Do the plot of accuracy
plt.plot(train_iter, train_accuracy,'b-', label='Training data (mini-batch)')
plt.plot(test_iter, test_accuracy,'r-', label='Testing data')
plt.xlabel('Iteration')
plt.ylabel('Prediction accuracy')
plt.ylim([max(1-(1-test_accuracy[-1])*2,0),1])
plt.title('Prediction accuracy')
plt.grid(True)
plt.legend(loc='best')
plt.show()

In [ ]:
# Evaluate on all test data
tmp = sess.run([prediction,P], feed_dict = {Xim: test_data})
test_pred = tmp[0]
test_P = tmp[1]
test_true = sess.run(true, feed_dict = {Y: test_labels})

# Close the session
sess.close()

In [ ]:
# Extract 100 random test images
batch_test_ind = np.random.choice(test_size,100)
batch_test_ind_wrong_first = batch_test_ind[np.argsort(test_true[batch_test_ind]==test_pred[batch_test_ind])]

# Do the plot of the images
num_rows = 10
num_cols = 10
num_images = num_rows*num_cols
plt.figure(figsize=(num_cols, num_rows))

for i, i_im in enumerate(batch_test_ind_wrong_first, 1):
    plt.subplot(num_rows, num_cols, i)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(test_data[i_im,:,:,0], cmap=plt.cm.binary)    
    if test_pred[i_im]==test_true[i_im]:
          plt.text(0, 25, test_pred[i_im], fontsize=25, color='blue')
    else:
          plt.text(0, 25, test_pred[i_im], fontsize=25, color='red')    
                  
plt.show()
